In [1]:
from loader import Loader
from indexer import DocumentIndexer
from retriever import Retriever
from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.base import Chain

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
urls=[
    "https://g1.globo.com/rs/rio-grande-do-sul/noticia/2024/05/15/levantamento-enchente-porto-alegre-bairros-e-moradores.ghtml",
    "https://www.cnnbrasil.com.br/nacional/enchente-em-porto-alegre-veja-imagens-da-cidade-apos-queda-no-nivel-da-agua/",
    "https://www.bbc.com/portuguese/articles/cw00d51k5rlo",
    "https://gauchazh.clicrbs.com.br/ultimas-noticias/tag/alagamentos/",
    "https://www.poder360.com.br/infraestrutura/especialistas-listam-medidas-para-evitar-enchentes-em-porto-alegre/"
]

#documents = Loader.load_documents(urls, chunk=True, chunk_model_name="gpt-3.5-turbo", chunk_size=500, chunk_overlap=50, save_path="documents.pkl")
documents = Loader.load_from_file("documents.pkl")

In [3]:
#db = DocumentIndexer.create_new_db(documents, persist_directory="./db")
db = DocumentIndexer.load_db("./db")

In [4]:
retriever = Retriever.create_retriever_from_db(db)

In [5]:
system_prompt: str = """
Você é um assistente de perguntas e respostas inserido em uma RAG (Retrieval-Augmented Generation). \
Você receberá um contexto extraído de documentos e uma pergunta feita pelo usuário. Sua tarefa é \
responder à pergunta usando exclusivamente o contexto fornecido. Se o contexto não for fornecido ou \
não for suficiente para responder à pergunta, responda com "Eu não sei". Utilize apenas as informações \
contidas no contexto fornecido.

Contexto fornecido: {context}
"""


prompt = ChatPromptTemplate.from_messages([("system", system_prompt),("human", "{input}")])
llm = ChatOpenAI(model="gpt-4o")

In [6]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [7]:
response = rag_chain.invoke({"input": "Qual a causa das enchentes em Porto Alegre?"})

In [10]:
response["answer"]

'A causa das enchentes em Porto Alegre é a falta de manutenção permanente nos sistemas de proteção contra inundações e de drenagem pluvial, especialmente nas comportas ao longo do Muro da Mauá, abaixo da Avenida Castelo Branco, e nas comportas junto às Casas de Bombas.'

In [8]:
for i in response["context"]:
    print(i.page_content)

longo prazo, não tem o que fazer. Teremos de conviver com o sentimento de medo de estar ali, na cama, e ocorrer uma nova enchente”,  afirma.Além de interromper o sonho de normalização de muitos gaúchos, a enxurrada de sexta-feira (23/5) travou operações de reocupação de espaços públicos e destruiu obras emergenciais que serviam de paliativo diante da aniquilação da infraestrutura do Estado.Em Porto Alegre, a prefeitura havia programado o início da limpeza do Mercado Público. Com as chuvas, foi preciso adiar o trabalho.No bairro Sarandi, zona norte de Porto Alegre, a chuva e a correnteza do Arroio das Pedras derrubaram um trecho da Avenida Sarandi e do talude de contenção na esquina com a Avenida General Raphael Zippin. Embora o Sarandi seja o bairro onde a enchente atingiu o maior número de pessoas, com 26.042 pessoas afetadas segundo a prefeitura, esse ponto da avenida não havia sofrido danos até ontem.No Vale do Taquari, uma ponte provisória flutuante instalada pelo Exército sobre o 

In [11]:
response = rag_chain.invoke({"input": "Qual o verdadeiro nome do Darth Vader?"})
response["answer"]

'Eu não sei.'